In [122]:
import pandas as pd
import numpy as np
from functools import reduce

In [80]:
data = {'x1':[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3],'x2':['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L'],
       'Y':[-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]}

In [81]:
data = pd.DataFrame(data)

In [82]:
data

,x1,x2,Y
0,1,S,-1
1,1,M,-1
2,1,M,1
3,1,S,1
4,1,S,-1
5,2,S,-1
6,2,M,-1
7,2,M,1
8,2,L,1
9,2,L,1


In [5]:
p1 = pd.DataFrame(data[data['Y']==1].groupby('x1')['Y'].count()).T

In [9]:
p1.loc['p'] = p1.loc['Y']/sum(p1.loc['Y'])

In [11]:
p2 = pd.DataFrame(data[data['Y']==-1].groupby('x1')['Y'].count()).T

In [12]:
p2.loc['p'] = p2.loc['Y']/sum(p2.loc['Y'])

In [14]:
p3 = pd.DataFrame(data[data['Y']==1].groupby('x2')['Y'].count()).T

In [15]:
p3.loc['p'] = p3.loc['Y']/sum(p3.loc['Y'])

In [17]:
p4 = pd.DataFrame(data[data['Y']==-1].groupby('x2')['Y'].count()).T

In [18]:
p4.loc['p'] = p4.loc['Y']/sum(p4.loc['Y'])

In [19]:
p4

x2,L,M,S
Y,1.000000,2.000000,3.0
p,0.166667,0.333333,0.5


In [30]:
p1 = data[data['Y']==1]['x1'].value_counts()/data[data['Y']==1]['x1'].value_counts().sum()

In [31]:
p2 = data[data['Y']==1]['x2'].value_counts()/data[data['Y']==1]['x2'].value_counts().sum()

In [37]:
pd.DataFrame(pd.concat([p1,p2],axis=1).T.stack())

0
x1 1  0.222222
   2  0.333333
   3  0.444444
x2 L  0.444444
   M  0.444444
   S  0.111111

In [38]:
p3 = data[data['Y']==-1]['x1'].value_counts()/data[data['Y']==-1]['x1'].value_counts().sum()

In [39]:
p4 = data[data['Y']==-1]['x2'].value_counts()/data[data['Y']==-1]['x2'].value_counts().sum()

In [40]:
pd.DataFrame(pd.concat([p3,p4],axis=1).T.stack())

0
x1 1  0.500000
   2  0.333333
   3  0.166667
x2 L  0.166667
   M  0.333333
   S  0.500000

In [68]:
def frency(X:list,Y:list):
    fren = {}
    for y in Y:
        d = {}
        for x in X:
            p = data[data['Y']==y][x].value_counts()/data[data['Y']==y][x].value_counts().sum()
            d.update(p)
            f = {y:d}
        fren.update(f)
    return fren

In [69]:
X = ['x1','x2'];Y = [1,-1]

In [70]:
fr = frency(X,Y)

In [71]:
fr

{1: {3: 0.4444444444444444,
  2: 0.3333333333333333,
  1: 0.2222222222222222,
  'L': 0.4444444444444444,
  'M': 0.4444444444444444,
  'S': 0.1111111111111111},
 -1: {1: 0.5,
  2: 0.3333333333333333,
  3: 0.16666666666666666,
  'S': 0.5,
  'M': 0.3333333333333333,
  'L': 0.16666666666666666}}

In [96]:
def predict(fr,Y,feature1,feature2):
    com = []
    for i in Y:
        Py = (data['Y']==i).sum()/data.shape[0]
        Pz = Py*fr[i][feature1]*fr[i][feature2]
        com.append(Pz)
    return Y[com.index(max(com))]

In [97]:
predict(fr,Y,2,'S')

-1

In [161]:
class Bayes:
    def __init__(self,data,X,Y):
        """
        :data 数据集
        :X 特征
        :Y 标签值
        """
        self.data = data
        self.X = X
        self.Y = Y
        
    def frency(self):
        fren = {}
        for y in self.Y:
            d = {}
            for x in self.X:
                p = self.data[self.data['Y']==y][x].value_counts()/self.data[self.data['Y']==y][x].value_counts().sum()
                d.update(p)
                f = {y:d}
            fren.update(f)
        return fren
    
    def predict(self,features:list):
        """
        :features 需预测特征的值
        """
        fr = self.frency()
        com = []
        for i in self.Y:
            featurefr = [(self.data['Y']==i).sum()/self.data.shape[0]]+[fr[i][f] for f in features]
            pz = reduce(lambda x,y:x*y,featurefr)
            com.append(pz)
        return self.Y[com.index(max(com))]

In [162]:
bayes = bayes(data,['x1','x2'],[1,-1])

In [164]:
bayes.predict([2,'L'])

1